<a href="https://colab.research.google.com/github/LYULU2/covid-vaccine-misinformation/blob/master/CTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!pip install contextualized-topic-models
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U tqdm

## Restart the Notebook

In [1]:
import numpy as np
import pandas as pd
import string
from gensim import matutils, models
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction import text

import matplotlib.pyplot as plt
import seaborn as sns

from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Alt-Before

### read data

In [185]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [186]:
data_alt_before = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/alt_before_cleaned.csv', delimiter=',')
data_alt_before_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Alt-right-2018-01-01--2020-02-29.csv', delimiter=',')['Message'])

In [187]:
add_stop_words = ['said','say', 'says','news', 'year', 'vaccine', 'covid', 'people','COVID-19','COVID',
                  'coronavirus', 'vaccination', 'vaccinated', 'Pfizer','Pfizer-BioNTech',
                  'Moderna', 'Johnson', 'Johnson’s', 'Janssen','j&j', 'mRNA', 'vaccination',
                  'freedom', 'truth','science', 'government', 'control', 'dictate','personal',
                  'choice', 'hoax', 'flu','www','com', 'http', 'vaccin']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
spanish = nltk.corpus.stopwords.words('spanish')
stop_words = stop_words.union(spanish)

def clean_text(text):
  stemmed_text = []
  porter_stemmer = PorterStemmer()
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'www\.[a-z]+\.at', '', text)
  text = re.sub(r'(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) #remove mentions
  text = re.sub(r'bit.ly/\S+', '', text)
  text = re.sub(r'^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', '', text)
  text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text)
  result = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  result = result.lower()
  result = result.split()
  result = [r for r in result if r not in stop_words]
  stemmed_result = [porter_stemmer.stem(r) for r in result]
  stemmed_text.append(" ".join(stemmed_result))
  return stemmed_text

In [188]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(clean_text)
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return data

alt_before = pre_process_without_combine(data_alt_before_corpus)

In [189]:
data_alt_before = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/alt_before_cleaned.csv', delimiter=',')
data_alt_before_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Alt-right-2018-01-01--2020-02-29.csv', delimiter=',')['Message'])

In [190]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return pd.DataFrame(data['Message'])

In [191]:
data_alt_before_corpus = pre_process_without_combine(data_alt_before_corpus)

In [192]:
alt_before

,Message
1,happi birthday madelein carrol highest paid ac...
3,pelosi absolut lost mind
5,break
9,donald j trump democrat busi wast time impeach...
10,went bartend vocal voic left
...,...
4734,corrupt call corrupt wow world world defin won...
4735,stun result
4736,advertis decid warn m sure ll take soon
4741,fact weshallnotforget prideandheritageforward ...


In [193]:
alt_before = alt_before.reset_index()

In [194]:
data_alt_before_corpus = data_alt_before_corpus.reset_index()

In [195]:
data_alt_before_corpus.columns = ['index', 'Raw_Message']

In [196]:
alt_before_join = alt_before.merge(data_alt_before_corpus)
alt_before_join

,index,Message,Raw_Message
0,1,happi birthday madelein carrol highest paid ac...,Happy Birthday To Madeleine Carroll. She Was T...
1,3,pelosi absolut lost mind,Pelosi Has Absolutely Lost Her Mind! 😲😒
2,5,break,BREAKING
3,9,donald j trump democrat busi wast time impeach...,Donald J. Trump Do Nothing Democrats were busy...
4,10,went bartend vocal voic left,This from someone who went from bartender to o...
...,...,...,...
2621,4734,corrupt call corrupt wow world world defin won...,THE CORRUPT ARE CALLING OTHERS CORRUPT Wow!! W...
2622,4735,stun result,STUNNING RESULT!
2623,4736,advertis decid warn m sure ll take soon,Truth in Advertising....Someone decided to war...
2624,4741,fact weshallnotforget prideandheritageforward ...,Truth & Fact... #WeShallNotForget #PrideandHer...


In [197]:
alt_before_join.drop(columns = 'index',inplace=True)

In [198]:
alt_before_join

,Message,Raw_Message
0,happi birthday madelein carrol highest paid ac...,Happy Birthday To Madeleine Carroll. She Was T...
1,pelosi absolut lost mind,Pelosi Has Absolutely Lost Her Mind! 😲😒
2,break,BREAKING
3,donald j trump democrat busi wast time impeach...,Donald J. Trump Do Nothing Democrats were busy...
4,went bartend vocal voic left,This from someone who went from bartender to o...
...,...,...
2621,corrupt call corrupt wow world world defin won...,THE CORRUPT ARE CALLING OTHERS CORRUPT Wow!! W...
2622,stun result,STUNNING RESULT!
2623,advertis decid warn m sure ll take soon,Truth in Advertising....Someone decided to war...
2624,fact weshallnotforget prideandheritageforward ...,Truth & Fact... #WeShallNotForget #PrideandHer...


### ZeroShotTM

In [28]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v2")

training_dataset = tp.fit(text_for_contextual=alt_before_join['Raw_Message'], text_for_bow=alt_before_join['Message'])

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [29]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [39390/39390]	Train Loss: 682.351283960277	Time: 0:00:01.608123: : 15it [00:24,  1.64s/it]
Sampling: [20/20]: : 20it [00:26,  1.34s/it]


In [30]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['presid', 'continu', 'minist', 'zambia', 'make'],
             1: ['equal', 'cut', 'exercis', 'main', 'strengthen'],
             2: ['machineri', 'saharan', 'amnesti', 'chosen', 'proport'],
             3: ['time', 'mr', 'democrat', 'state', 'use'],
             4: ['trump', 'thing', 'liber', 'make', 'today'],
             5: ['ramirez', 'fee', 'boast', 'priorit', 'french'],
             6: ['html', 'america', 've', 'refus', 'messag'],
             7: ['dress', 'commerci', 'beam', 'friendli', 'collar'],
             8: ['god', 'stand', 'call', 'stori', 'local'],
             9: ['messag', 'east', 'read', 'john', 'html'],
             10: ['couldn', 'excess', 'mayb', 'smallhold', 'broad'],
             11: ['lebohang', 'imperson', 'cosponsor', 'readmit', 'xkiqeq'],
             12: ['click', 'index', 'literatur', 'alamoministri', 'html'],
             13: ['grate', 'bare', 'wise', 'scientist', 'mutual'],
             14: ['resent', 'unfair', 'sphere',

In [32]:
ctm.get_doc_topic_distribution(training_dataset, n_samples=20)

Sampling: [20/20]: : 20it [00:26,  1.33s/it]


array([[0.01052515, 0.00897337, 0.01186503, ..., 0.00591496, 0.01107789,
        0.0180976 ],
       [0.00492709, 0.02168097, 0.03139576, ..., 0.03432643, 0.00360788,
        0.0052426 ],
       [0.00445955, 0.02383865, 0.08038545, ..., 0.10138373, 0.00367877,
        0.00219151],
       ...,
       [0.0125275 , 0.02716107, 0.0218707 , ..., 0.02179904, 0.00992544,
        0.00576406],
       [0.00576102, 0.01360021, 0.01794332, ..., 0.01165518, 0.00323504,
        0.00258107],
       [0.00506683, 0.021872  , 0.01786223, ..., 0.02335826, 0.01982814,
        0.00349581]])

In [33]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [39390/39390]	Train Loss: 667.0836761980137	Time: 0:00:01.621543: : 15it [00:24,  1.62s/it]
0it [08:37, ?it/s]
0it [06:16, ?it/s]
Sampling: [20/20]: : 20it [00:27,  1.38s/it]


In [34]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['craft', 'kabeta', 'illiber', 'shareabl', 'deviou'],
             1: ['scream', 'enrich', 'height', 'absent', 'mindset'],
             2: ['craft', 'fisheri', 'mount', 'handcuf', 'curtail'],
             3: ['commun', 'presid', 'state', 'respons', 'nation'],
             4: ['theft', 'equival', 'articul', 'steve', 'emul'],
             5: ['trump', 'day', 'democrat', 'end', 'told'],
             6: ['presid', 'parti', 'mr', 'leader', 'need'],
             7: ['rent', 'requisit', 'kabeta', 'curtail', 'illiber'],
             8: ['html', 'english', 'literatur', 'read', 'alamoministri'],
             9: ['major', 'day', 'thing', 'did', 'time'],
             10: ['zambia', 'develop', 'countri', 'continu', 'zambian'],
             11: ['trump', 'didn', 'democrat', 'immigr', 'american'],
             12: ['africa', 'african', 'lungu', 'presid', 'countri'],
             13: ['alexand', 'handcuf', 'craft', 'kabeta', 'rainbow'],
             14: ['corner', 'su

In [51]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [52]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,craft kabeta illiber shareabl deviou
1,scream enrich height absent mindset
2,craft fisheri mount handcuf curtail
3,commun presid state respons nation
4,theft equival articul steve emul
5,trump day democrat end told
6,presid parti mr leader need
7,rent requisit kabeta curtail illiber
8,html english literatur read alamoministri
9,major day thing did time


### Combined Topic Model

In [53]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [39390/39390]	Train Loss: 667.536564102812	Time: 0:00:01.828797: : 15it [00:27,  1.85s/it]
Sampling: [20/20]: : 20it [00:29,  1.45s/it]


In [54]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['intens', 'oversight', 'suit', 'larger', 'overt'],
             1: ['websit', 'gospel', 'literatur', 'bunch', 'english'],
             2: ['mr', 'nation', 'includ', 'state', 'zambia'],
             3: ['tmksf', 'rig', 'supervisor', 'ngo', 'safaa'],
             4: ['zambia', 'countri', 'nation', 'member', 'edgar'],
             5: ['sympathi', 'stadium', 'recov', 'hesit', 'align'],
             6: ['devast', 'ngo', 'sincer', 'provok', 'frequenc'],
             7: ['colombia', 'lung', 'kachingw', 'zeep', 'tmksf'],
             8: ['friendli', 'consensu', 'shop', 'percentag', 'mortal'],
             9: ['nationalist', 'kogali', 'zeep', 'certifi', 'saavedra'],
             10: ['tone', 'sincer', 'nonsens', 'excus', 'exacerb'],
             11: ['zambia', 'develop', 'nation', 'countri', 'manag'],
             12: ['fond', 'prepared', 'technocrat', 'nypost', 'ngo'],
             13: ['mutatui', 'copycat', 'fattah', 'repetit', 'agreess'],
             14: [

In [55]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [56]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,intens oversight suit larger overt
1,websit gospel literatur bunch english
2,mr nation includ state zambia
3,tmksf rig supervisor ngo safaa
4,zambia countri nation member edgar
5,sympathi stadium recov hesit align
6,devast ngo sincer provok frequenc
7,colombia lung kachingw zeep tmksf
8,friendli consensu shop percentag mortal
9,nationalist kogali zeep certifi saavedra


In [ ]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['craft', 'kabeta', 'illiber', 'shareabl', 'deviou'],
             1: ['scream', 'enrich', 'height', 'absent', 'mindset'],
             2: ['craft', 'fisheri', 'mount', 'handcuf', 'curtail'],
             3: ['commun', 'presid', 'state', 'respons', 'nation'],
             4: ['theft', 'equival', 'articul', 'steve', 'emul'],
             5: ['trump', 'day', 'democrat', 'end', 'told'],
             6: ['presid', 'parti', 'mr', 'leader', 'need'],
             7: ['rent', 'requisit', 'kabeta', 'curtail', 'illiber'],
             8: ['html', 'english', 'literatur', 'read', 'alamoministri'],
             9: ['major', 'day', 'thing', 'did', 'time'],
             10: ['zambia', 'develop', 'countri', 'continu', 'zambian'],
             11: ['trump', 'didn', 'democrat', 'immigr', 'american'],
             12: ['africa', 'african', 'lungu', 'presid', 'countri'],
             13: ['alexand', 'handcuf', 'craft', 'kabeta', 'rainbow'],
             14: ['corner', 'su

In [201]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=10, num_epochs=15)
ctm.fit(training_dataset) # run the model
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

Epoch: [15/15]	 Seen Samples: [80565/80565]	Train Loss: 305.025355351657	Time: 0:00:04.117292: : 15it [01:01,  4.12s/it]
Sampling: [20/20]: : 20it [00:59,  2.98s/it]


In [202]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,soc sh twtr tsrc yahoo
1,hogi olt az nem sz
2,lou bud stori facebook mask
3,case mandat state health report
4,telegram dwt older imss zone
5,abandon cancelar gobernaci movil amayrani
6,childrenshealthdefen rez hate mutat cghixf
7,vacuna yang dan susah rakyat
8,approv pandem page start die
9,just human york new claim


## Alt-After

### read data

In [170]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/alt_after_cleaned.csv', delimiter=',')
data_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Alt-right-2020-03-01--2021-09-19.csv', delimiter=',')['Message'])

In [171]:
add_stop_words = ['said','say', 'says','news', 'year', 'vaccine', 'covid', 'people','COVID-19','COVID',
                  'coronavirus', 'vaccination', 'vaccinated', 'Pfizer','Pfizer-BioNTech',
                  'Moderna', 'Johnson', 'Johnson’s', 'Janssen','j&j', 'mRNA', 'vaccination',
                  'freedom', 'truth','science', 'government', 'control', 'dictate','personal',
                  'choice', 'hoax', 'flu','www','com', 'http', 'vaccin']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
spanish = nltk.corpus.stopwords.words('spanish')
stop_words = stop_words.union(spanish)

def clean_text(text):
  stemmed_text = []
  porter_stemmer = PorterStemmer()
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'www\.[a-z]+\.at', '', text)
  text = re.sub(r'(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) #remove mentions
  text = re.sub(r'bit.ly/\S+', '', text)
  text = re.sub(r'^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', '', text)
  text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text)
  result = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  result = result.lower()
  result = result.split()
  result = [r for r in result if r not in stop_words]
  stemmed_result = [porter_stemmer.stem(r) for r in result]
  stemmed_text.append(" ".join(stemmed_result))
  return stemmed_text

In [172]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(clean_text)
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return data

clean = pre_process_without_combine(data)

In [173]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/alt_after_cleaned.csv', delimiter=',')
data_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Alt-right-2020-03-01--2021-09-19.csv', delimiter=',')['Message'])

In [174]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return pd.DataFrame(data['Message'])

In [175]:
data_corpus = pre_process_without_combine(data_corpus)

In [176]:
data_corpus

,Message
1,Classified testimony in opposition to the Bide...
2,UN GENERAL ASSEMBLY WILL MOSTLY BE VIRTUAL The...
3,GERMANY NO LONGER REGARDS ZAMBIA AS A COVID-19...
4,Why are they dividing people like this?
5,Mornin' Rebels... Like our Page to see more!
...,...
9321,Super Tuesday Is Tomorrow But In A Surprising ...
9322,Funny how the media treats this differently no...
9323,"Just a reminder that it took 1,000 deaths befo..."
9325,THE MEDIA DOESN'T WANT YOU TO SHARE THIS


In [177]:
clean = clean.reset_index()

In [178]:
data_corpus = data_corpus.reset_index()

In [179]:
data_corpus.columns = ['index', 'Raw_Message']

In [180]:
join = clean.merge(data_corpus)
join

,index,Message,Raw_Message
0,1,gener assembl virtual unit state request unit ...,Classified testimony in opposition to the Bide...
1,2,germani longer regard zambia riski area excel ...,UN GENERAL ASSEMBLY WILL MOSTLY BE VIRTUAL The...
2,3,divid like,GERMANY NO LONGER REGARDS ZAMBIA AS A COVID-19...
3,4,mornin rebel like page,Why are they dividing people like this?
4,5,wow,Mornin' Rebels... Like our Page to see more!
...,...,...,...
2976,5356,presid trump wow support north carolina corona...,🔴 ATTENTION! 🔴 The owner of United Deplorables...
2977,5358,advocaci director max mcguir oan night talk ba...,President Trump marks Veterans Day 2020 at Arl...
2978,5360,super tuesday tomorrow surpri aipac speech tod...,Can’t even make this up
2979,5363,media doesn t want share,This needs to be shared by all members on this...


In [181]:
join.drop(columns = 'index',inplace=True)

In [182]:
join

,Message,Raw_Message
0,gener assembl virtual unit state request unit ...,Classified testimony in opposition to the Bide...
1,germani longer regard zambia riski area excel ...,UN GENERAL ASSEMBLY WILL MOSTLY BE VIRTUAL The...
2,divid like,GERMANY NO LONGER REGARDS ZAMBIA AS A COVID-19...
3,mornin rebel like page,Why are they dividing people like this?
4,wow,Mornin' Rebels... Like our Page to see more!
...,...,...
2976,presid trump wow support north carolina corona...,🔴 ATTENTION! 🔴 The owner of United Deplorables...
2977,advocaci director max mcguir oan night talk ba...,President Trump marks Veterans Day 2020 at Arl...
2978,super tuesday tomorrow surpri aipac speech tod...,Can’t even make this up
2979,media doesn t want share,This needs to be shared by all members on this...


### ZeroShotTM

In [80]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v2")

training_dataset = tp.fit(text_for_contextual=join['Raw_Message'], text_for_bow=join['Message'])

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [81]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [44715/44715]	Train Loss: 965.4041193181819	Time: 0:00:01.917896: : 15it [00:28,  1.90s/it]
Sampling: [20/20]: : 20it [00:31,  1.56s/it]


In [82]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['commun', 'day', 'play', 'associ', 'forc'],
             1: ['futur', 'polici', 'kahl', 'demand', 'militari'],
             2: ['like', 'code', 'provinc', 'just', 'state'],
             3: ['zambia', 'speech', 'leav', 'public', 'countri'],
             4: ['zambia', 'allow', 'countri', 'mr', 'hon'],
             5: ['nicaragua', 'ortega', 'violeta', 'cristiana', 'suburban'],
             6: ['st', 'children', 'good', 'war', 'confirm'],
             7: ['monument', 'trump', 'women', 'octob', 'time'],
             8: ['protect', 'prevent', 'human', 'china', 'effort'],
             9: ['somoza', 'abnorm', 'managua', 'pretti', 'nicaragua'],
             10: ['north', 'iran', 'deal', 'univ', 'debat'],
             11: ['death', 'just', 'democraci', 'north', 'record'],
             12: ['assign', 'limit', 'lose', 'mistak', 'sentenc'],
             13: ['protect', 'state', 'account', 'congress', 'crisi'],
             14: ['result', 'pandem', 'zambia', 'offi

In [83]:
ctm.get_doc_topic_distribution(training_dataset, n_samples=20)

Sampling: [20/20]: : 20it [00:31,  1.57s/it]


array([[0.00357267, 0.01441412, 0.01039579, ..., 0.00770453, 0.00621199,
        0.00343547],
       [0.01328897, 0.00770177, 0.01041165, ..., 0.00290565, 0.00674915,
        0.00235404],
       [0.0052313 , 0.06232346, 0.00421992, ..., 0.00509632, 0.00325132,
        0.00803141],
       ...,
       [0.02277823, 0.00442161, 0.01279623, ..., 0.02270307, 0.02118252,
        0.01550149],
       [0.02068   , 0.02528542, 0.01877489, ..., 0.01493922, 0.01564527,
        0.02771954],
       [0.02217861, 0.02153281, 0.02302172, ..., 0.01464125, 0.02755107,
        0.01134152]])

In [84]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [44715/44715]	Train Loss: 952.8863544636866	Time: 0:00:01.872477: : 15it [00:28,  1.91s/it]
Sampling: [20/20]: : 20it [00:31,  1.58s/it]


In [85]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['bit', 'daili', 'facebook', 'googl', 'conserv'],
             1: ['googl', 'avail', 'help', 'support', 'facebook'],
             2: ['daili', 'bit', 'podcast', 'conserv', 'ly'],
             3: ['like', 'daili', 'podcast', 'conserv', 'bit'],
             4: ['economi', 'school', 'mr', 'infrastructur', 'hou'],
             5: ['iceland', 'schooler', 'biotech', 'motorcad', 'enthusiasm'],
             6: ['iceland', 'schooler', 'biotech', 'worth', 'spotlight'],
             7: ['presid', 'need', 'job', 'know', 'elect'],
             8: ['nation', 'today', 'john', 'martin', 'black'],
             9: ['iceland', 'janel', 'mart', 'motorcad', 'schooler'],
             10: ['john', 'martin', 'watch', 'campaign', 'talk'],
             11: ['biotech', 'schooler', 'iceland', 'gofundm', 'secretserv'],
             12: ['happen', 'chanda', 'district', 'pm', 'concern'],
             13: ['congress', 'deal', 'tell', 'episod', 'african'],
             14: ['work', 'a

In [86]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [87]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,bit daili facebook googl conserv
1,googl avail help support facebook
2,daili bit podcast conserv ly
3,like daili podcast conserv bit
4,economi school mr infrastructur hou
5,iceland schooler biotech motorcad enthusiasm
6,iceland schooler biotech worth spotlight
7,presid need job know elect
8,nation today john martin black
9,iceland janel mart motorcad schooler


### Combined Topic Model

In [88]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [44715/44715]	Train Loss: 886.1428631331768	Time: 0:00:02.043325: : 15it [00:30,  2.04s/it]
Sampling: [20/20]: : 20it [00:32,  1.61s/it]


In [89]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['liber', 'journalist', 'penc', 'doe', 'care'],
             1: ['ly', 'conserv', 'rumbl', 'join', 'daili'],
             2: ['citi', 'close', 'month', 'york', 'told'],
             3: ['rich', 'signatur', 'countrywid', 'heavili', 'kamba'],
             4: ['zambia', 'law', 'polit', 'presid', 'pf'],
             5: ['johnmartintalk', 'org', 'talk', 'martin', 'john'],
             6: ['ly', 'daili', 'conserv', 'podcast', 'text'],
             7: ['zambia', 'continu', 'parti', 'pf', 'public'],
             8: ['zambia', 'presid', 'countri', 'nation', 'lungu'],
             9: ['podcast', 'conserv', 'daili', 'ly', 'cd'],
             10: ['intellig', 'staff', 'author', 'visit', 'world'],
             11: ['outlook', 'distinguish', 'wonder', 'av', 'rapucsrx'],
             12: ['conserv', 'daili', 'like', 'ly', 'podcast'],
             13: ['czggemlyygivfrgyqdefi',
              'hdi',
              'uya',
              'pusbsgnxvxvl',
              'zgnlm

In [90]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [91]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,liber journalist penc doe care
1,ly conserv rumbl join daili
2,citi close month york told
3,rich signatur countrywid heavili kamba
4,zambia law polit presid pf
5,johnmartintalk org talk martin john
6,ly daili conserv podcast text
7,zambia continu parti pf public
8,zambia presid countri nation lungu
9,podcast conserv daili ly cd


In [92]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['liber', 'journalist', 'penc', 'doe', 'care'],
             1: ['ly', 'conserv', 'rumbl', 'join', 'daili'],
             2: ['citi', 'close', 'month', 'york', 'told'],
             3: ['rich', 'signatur', 'countrywid', 'heavili', 'kamba'],
             4: ['zambia', 'law', 'polit', 'presid', 'pf'],
             5: ['johnmartintalk', 'org', 'talk', 'martin', 'john'],
             6: ['ly', 'daili', 'conserv', 'podcast', 'text'],
             7: ['zambia', 'continu', 'parti', 'pf', 'public'],
             8: ['zambia', 'presid', 'countri', 'nation', 'lungu'],
             9: ['podcast', 'conserv', 'daili', 'ly', 'cd'],
             10: ['intellig', 'staff', 'author', 'visit', 'world'],
             11: ['outlook', 'distinguish', 'wonder', 'av', 'rapucsrx'],
             12: ['conserv', 'daili', 'like', 'ly', 'podcast'],
             13: ['czggemlyygivfrgyqdefi',
              'hdi',
              'uya',
              'pusbsgnxvxvl',
              'zgnlm

In [183]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=10, num_epochs=15)
ctm.fit(training_dataset) # run the model
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

Epoch: [15/15]	 Seen Samples: [80565/80565]	Train Loss: 309.5793499304715	Time: 0:00:04.068562: : 15it [01:01,  4.11s/it]
Sampling: [20/20]: : 20it [00:58,  2.92s/it]


In [184]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,cancer gene autoridad travel camp
1,busi small love facebook googl
2,san hospit massiv fda despit
3,need health like new employ
4,hou employ home sick care
5,sz az hogi azt egi
6,zero thought transport memo effort
7,team modern pregnant question level
8,entrust contact unelect pursuit invalid
9,invalid abu entrust unelect wuhan


## anti-vaxxer-Before

### read data

In [154]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/anti_before_cleaned.csv', delimiter=',')
data_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/AVaxxer-2018-01-01--2020-02-29.csv', delimiter=',')['Message'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [155]:
add_stop_words = ['said','say', 'says','news', 'year', 'vaccine', 'covid', 'people','COVID-19','COVID',
                  'coronavirus', 'vaccination', 'vaccinated', 'Pfizer','Pfizer-BioNTech',
                  'Moderna', 'Johnson', 'Johnson’s', 'Janssen','j&j', 'mRNA', 'vaccination',
                  'freedom', 'truth','science', 'government', 'control', 'dictate','personal',
                  'choice', 'hoax', 'flu','www','com', 'http', 'vaccin']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
spanish = nltk.corpus.stopwords.words('spanish')
stop_words = stop_words.union(spanish)

def clean_text(text):
  stemmed_text = []
  porter_stemmer = PorterStemmer()
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'www\.[a-z]+\.at', '', text)
  text = re.sub(r'(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) #remove mentions
  text = re.sub(r'bit.ly/\S+', '', text)
  text = re.sub(r'^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', '', text)
  text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text)
  result = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  result = result.lower()
  result = result.split()
  result = [r for r in result if r not in stop_words]
  stemmed_result = [porter_stemmer.stem(r) for r in result]
  stemmed_text.append(" ".join(stemmed_result))
  return stemmed_text

In [156]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(clean_text)
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return data

clean = pre_process_without_combine(data)

In [157]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/anti_before_cleaned.csv', delimiter=',')
data_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/AVaxxer-2018-01-01--2020-02-29.csv', delimiter=',')['Message'])

In [158]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return pd.DataFrame(data['Message'])

In [159]:
data_corpus = pre_process_without_combine(data_corpus)

In [160]:
data_corpus

,Message
0,THIS IS WHAT'S REALLY GONNA KILL EVERYBODY!!!!...
1,DMT is the spiritual science which allows us t...
2,Thoughts? Opinions?
3,#Auspol Unbelievable how are they getting away...
4,When a woman asked Buttigieg how he would hand...
...,...
62466,Elevated glucose in pregnancy - even in non-di...
62467,Harrowing Predictions of Hope | Listen to toda...
62468,Rewarded for lying.
62469,Science is never settled. https://anewkindofhu...


In [161]:
clean = clean.reset_index()

In [162]:
data_corpus = data_corpus.reset_index()

In [163]:
data_corpus.columns = ['index', 'Raw_Message']

In [164]:
join = clean.merge(data_corpus)
join

,index,Message,Raw_Message
0,0,s realli gonna kill everybodi,THIS IS WHAT'S REALLY GONNA KILL EVERYBODY!!!!...
1,1,dmt spiritu allow traver gap matter consciou f...,DMT is the spiritual science which allows us t...
2,2,thought opinion,Thoughts? Opinions?
3,3,auspol unbeliev away garbag worker act sane pe...,#Auspol Unbelievable how are they getting away...
4,4,woman ask buttigieg handl debat thing like fai...,When a woman asked Buttigieg how he would hand...
...,...,...,...
34930,47304,worst sanofi s dengu launch turn nightmar note...,Why Is This Legal? Presentation on Vaccines by...
34931,47305,guarant big numerolog perspect number vibrat i...,Just think about it... The Truth About Vaccine...
34932,47306,review cour nation s histori face inflect poin...,"""The study says 7.7, for those of you counting..."
34933,47308,elev gluco pregnanc non diabet women increa bi...,"Watch the Democrats try to steal elections, an..."


In [165]:
join.drop(columns = 'index',inplace=True)

In [166]:
join

,Message,Raw_Message
0,s realli gonna kill everybodi,THIS IS WHAT'S REALLY GONNA KILL EVERYBODY!!!!...
1,dmt spiritu allow traver gap matter consciou f...,DMT is the spiritual science which allows us t...
2,thought opinion,Thoughts? Opinions?
3,auspol unbeliev away garbag worker act sane pe...,#Auspol Unbelievable how are they getting away...
4,woman ask buttigieg handl debat thing like fai...,When a woman asked Buttigieg how he would hand...
...,...,...
34930,worst sanofi s dengu launch turn nightmar note...,Why Is This Legal? Presentation on Vaccines by...
34931,guarant big numerolog perspect number vibrat i...,Just think about it... The Truth About Vaccine...
34932,review cour nation s histori face inflect poin...,"""The study says 7.7, for those of you counting..."
34933,elev gluco pregnanc non diabet women increa bi...,"Watch the Democrats try to steal elections, an..."


### ZeroShotTM

In [108]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v2")

training_dataset = tp.fit(text_for_contextual=join['Raw_Message'], text_for_bow=join['Message'])

Batches:   0%|          | 0/175 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [109]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [524025/524025]	Train Loss: 529.0124969810005	Time: 0:00:35.732306: : 15it [08:56, 35.74s/it]
Sampling: [20/20]: : 20it [08:51, 26.56s/it]


In [110]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['okleg', 'unusu', 'saturday', 'ingest', 'ty'],
             1: ['strip', 'sheath', 'myelin', 'encephalomyel', 'venezuela'],
             2: ['child', 'doctor', 'know', 'just', 'famili'],
             3: ['chemic', 'similar', 'experienc', 'older', 'respond'],
             4: ['time', 'word', 'lord', 'come', 'just'],
             5: ['wane', 'informedcon', 'gardasil', 'controversi', 'decrea'],
             6: ['detent', 'pfizer', 'childrenshealthdefen', 'comput', 'fro'],
             7: ['wednesday', 'wuhan', 'stake', 'bbc', 'judiciari'],
             8: ['implic', 'quarantin', 'coercion', 'bulli', 'suppli'],
             9: ['test', 'safeti', 'famili', 'risk', 'studi'],
             10: ['link', 'injuri', 'adver', 'author', 'sourc'],
             11: ['studi', 'protect', 'child', 'doctor', 'day'],
             12: ['pnn', 'tragic', 'causal', 'multido', 'rebut'],
             13: ['postpubert', 'dyscrasia', 'reconstitut', 'infr', 'tread'],
             

In [111]:
ctm.get_doc_topic_distribution(training_dataset, n_samples=20)

Sampling: [20/20]: : 20it [08:49, 26.49s/it]


array([[0.09721872, 0.07379208, 0.00117853, ..., 0.05165732, 0.00135792,
        0.00160175],
       [0.01248616, 0.00762436, 0.00498297, ..., 0.01412889, 0.00489426,
        0.00250257],
       [0.02407513, 0.04401648, 0.00351935, ..., 0.0560751 , 0.01517342,
        0.0056163 ],
       ...,
       [0.03878159, 0.0321567 , 0.01038148, ..., 0.04098053, 0.00862283,
        0.00618833],
       [0.0165073 , 0.0167284 , 0.01665848, ..., 0.01846164, 0.01978228,
        0.01539457],
       [0.03035986, 0.02263598, 0.01056955, ..., 0.01256345, 0.05194228,
        0.00757079]])

In [112]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [524025/524025]	Train Loss: 519.6319352010877	Time: 0:00:35.280381: : 15it [08:47, 35.19s/it]
Sampling: [20/20]: : 20it [08:44, 26.25s/it]


In [113]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['data', 'iwar', 'fbclid', 'tax', 'au'],
             1: ['state', 'don', 'day', 'like', 'god'],
             2: ['dysfunct', 'oxygen', 'effici', 'pelosi', 'ma'],
             3: ['childrenshealthdefen',
              'cdcwhistleblow',
              'va',
              'childrenshealth',
              'informedcon'],
             4: ['cid',
              'vaccineswork',
              'vaccinesharm',
              'tasherasimmon',
              'stopmandatoryvaccin'],
             5: ['utm', 'file', 'won', 'capitol', 'award'],
             6: ['god', 'faith', 'know', 'love', 'christ'],
             7: ['polio',
              'favorit',
              'informedchoicewa',
              'heartbreak',
              'vaccineimpact'],
             8: ['elev', 'exley', 'calvari', 'magnet', 'gap'],
             9: ['restrain', 'idaho', 'nvicadvocaci', 'snatch', 'troop'],
             10: ['van', 'remot', 'baker', 'changchun', 'beij'],
             11: ['place', 

In [114]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [115]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,data iwar fbclid tax au
1,state don day like god
2,dysfunct oxygen effici pelosi ma
3,childrenshealthdefen cdcwhistleblow va childre...
4,cid vaccineswork vaccinesharm tasherasimmon st...
5,utm file won capitol award
6,god faith know love christ
7,polio favorit informedchoicewa heartbreak vacc...
8,elev exley calvari magnet gap
9,restrain idaho nvicadvocaci snatch troop


### Combined Topic Model

In [116]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [524025/524025]	Train Loss: 493.8479978442107	Time: 0:00:59.163901: : 15it [14:49, 59.28s/it]
Sampling: [20/20]: : 20it [09:11, 27.59s/it]


In [117]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['cchfreedom',
              'hfm',
              'homepag',
              'tfvc',
              'childrenshealthdefen'],
             1: ['case', 'measl', 'know', 'children', 'good'],
             2: ['sanctiti', 'notabl', 'prospect', 'sentiment', 'taxat'],
             3: ['learntherisk', 'iwar', 'fbclid', 'hfm', 'homepag'],
             4: ['parent', 'don', 'follow', 'inform', 'religi'],
             5: ['steel', 'costli', 'ocr', 'portray', 'oha'],
             6: ['childrenshealthdefen',
              'cdcwhistleblow',
              'tfvc',
              'ly',
              'parentscalltheshot'],
             7: ['want', 'report', 'health', 'time', 'god'],
             8: ['come', 'gov', 'public', 'like', 'power'],
             9: ['genero', 'exacerb', 'tonight', 'pbe', 'hive'],
             10: ['gui', 'racial', 'usp', 'bomb', 'band'],
             11: ['health', 'use', 'state', 'children', 'famili'],
             12: ['state', 'don', 'know', 'com

In [118]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [119]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,cchfreedom hfm homepag tfvc childrenshealthdefen
1,case measl know children good
2,sanctiti notabl prospect sentiment taxat
3,learntherisk iwar fbclid hfm homepag
4,parent don follow inform religi
5,steel costli ocr portray oha
6,childrenshealthdefen cdcwhistleblow tfvc ly pa...
7,want report health time god
8,come gov public like power
9,genero exacerb tonight pbe hive


In [120]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['cchfreedom',
              'hfm',
              'homepag',
              'tfvc',
              'childrenshealthdefen'],
             1: ['case', 'measl', 'know', 'children', 'good'],
             2: ['sanctiti', 'notabl', 'prospect', 'sentiment', 'taxat'],
             3: ['learntherisk', 'iwar', 'fbclid', 'hfm', 'homepag'],
             4: ['parent', 'don', 'follow', 'inform', 'religi'],
             5: ['steel', 'costli', 'ocr', 'portray', 'oha'],
             6: ['childrenshealthdefen',
              'cdcwhistleblow',
              'tfvc',
              'ly',
              'parentscalltheshot'],
             7: ['want', 'report', 'health', 'time', 'god'],
             8: ['come', 'gov', 'public', 'like', 'power'],
             9: ['genero', 'exacerb', 'tonight', 'pbe', 'hive'],
             10: ['gui', 'racial', 'usp', 'bomb', 'band'],
             11: ['health', 'use', 'state', 'children', 'famili'],
             12: ['state', 'don', 'know', 'com

In [167]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=10, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [80565/80565]	Train Loss: 306.21717815438234	Time: 0:00:04.050897: : 15it [01:01,  4.07s/it]
Sampling: [20/20]: : 20it [00:58,  2.91s/it]


In [168]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [169]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,state health medic use infect
1,und von mero dico zu
2,stori facebook id fbid php
3,legislatur moderna serv est injur
4,subpoena western patmo prayer vrijheid
5,know stop just come like
6,patmo vrijheid cope manchest hcnradio
7,protest thursday tribun ich estat
8,podcast hogi si propheci mondta
9,worker children school new test


## anti-vaxxer-After

### read data

In [123]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/anti_after_cleaned.csv', delimiter=',')
data_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Alt-right-2020-03-01--2021-09-19.csv', delimiter=',')['Message'])

In [124]:
add_stop_words = ['said','say', 'says','news', 'year', 'vaccine', 'covid', 'people','COVID-19','COVID',
                  'coronavirus', 'vaccination', 'vaccinated', 'Pfizer','Pfizer-BioNTech',
                  'Moderna', 'Johnson', 'Johnson’s', 'Janssen','j&j', 'mRNA', 'vaccination',
                  'freedom', 'truth','science', 'government', 'control', 'dictate','personal',
                  'choice', 'hoax', 'flu','www','com', 'http', 'vaccin']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
spanish = nltk.corpus.stopwords.words('spanish')
stop_words = stop_words.union(spanish)

def clean_text(text):
  stemmed_text = []
  porter_stemmer = PorterStemmer()
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'www\.[a-z]+\.at', '', text)
  text = re.sub(r'(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) #remove mentions
  text = re.sub(r'bit.ly/\S+', '', text)
  text = re.sub(r'^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', '', text)
  text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text)
  result = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  result = result.lower()
  result = result.split()
  result = [r for r in result if r not in stop_words]
  stemmed_result = [porter_stemmer.stem(r) for r in result]
  stemmed_text.append(" ".join(stemmed_result))
  return stemmed_text

In [125]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(clean_text)
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return data

clean = pre_process_without_combine(data)

In [126]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/anti_after_cleaned.csv', delimiter=',')
data_corpus = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Alt-right-2020-03-01--2021-09-19.csv', delimiter=',')['Message'])

In [127]:
def pre_process_without_combine(data):
  data['Message'] = data['Message'].drop_duplicates()
  data['Message'] = data['Message'].fillna('')
  data['Message'] = data['Message'].astype('string')
  data['Message'] = data['Message'].apply(np.squeeze)
  data = data[data['Message']!=""]
  
  return pd.DataFrame(data['Message'])

In [128]:
data_corpus = pre_process_without_combine(data_corpus)

In [129]:
data_corpus

,Message
1,Classified testimony in opposition to the Bide...
2,UN GENERAL ASSEMBLY WILL MOSTLY BE VIRTUAL The...
3,GERMANY NO LONGER REGARDS ZAMBIA AS A COVID-19...
4,Why are they dividing people like this?
5,Mornin' Rebels... Like our Page to see more!
...,...
9321,Super Tuesday Is Tomorrow But In A Surprising ...
9322,Funny how the media treats this differently no...
9323,"Just a reminder that it took 1,000 deaths befo..."
9325,THE MEDIA DOESN'T WANT YOU TO SHARE THIS


In [130]:
clean = clean.reset_index()

In [131]:
data_corpus = data_corpus.reset_index()

In [132]:
data_corpus.columns = ['index', 'Raw_Message']

In [133]:
join = clean.merge(data_corpus)
join

,index,Message,Raw_Message
0,1,resumendeld editorialrv gobierno feder recateg...,Classified testimony in opposition to the Bide...
1,2,momento pastor iglesia vi mario toma barahona ...,UN GENERAL ASSEMBLY WILL MOSTLY BE VIRTUAL The...
2,4,noodelici,Why are they dividing people like this?
3,5,thousand came capitol salem oregon today despi...,Mornin' Rebels... Like our Page to see more!
4,7,wed vaccinechoicecanada,Wow.
...,...,...,...
5366,9321,protest plan place build calgari edmonton,Super Tuesday Is Tomorrow But In A Surprising ...
5367,9322,jab super spreader doesn t pictur just need f ...,Funny how the media treats this differently no...
5368,9323,reli maradiaga hace fuert llamado secretaria e...,"Just a reminder that it took 1,000 deaths befo..."
5369,9325,parent shot public charter school state texa a...,THE MEDIA DOESN'T WANT YOU TO SHARE THIS


In [134]:
join.drop(columns = 'index',inplace=True)

In [135]:
join

,Message,Raw_Message
0,resumendeld editorialrv gobierno feder recateg...,Classified testimony in opposition to the Bide...
1,momento pastor iglesia vi mario toma barahona ...,UN GENERAL ASSEMBLY WILL MOSTLY BE VIRTUAL The...
2,noodelici,Why are they dividing people like this?
3,thousand came capitol salem oregon today despi...,Mornin' Rebels... Like our Page to see more!
4,wed vaccinechoicecanada,Wow.
...,...,...
5366,protest plan place build calgari edmonton,Super Tuesday Is Tomorrow But In A Surprising ...
5367,jab super spreader doesn t pictur just need f ...,Funny how the media treats this differently no...
5368,reli maradiaga hace fuert llamado secretaria e...,"Just a reminder that it took 1,000 deaths befo..."
5369,parent shot public charter school state texa a...,THE MEDIA DOESN'T WANT YOU TO SHARE THIS


### ZeroShotTM

In [136]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v2")

training_dataset = tp.fit(text_for_contextual=join['Raw_Message'], text_for_bow=join['Message'])

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [137]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [80565/80565]	Train Loss: 324.53247638504234	Time: 0:00:03.523023: : 15it [00:53,  3.55s/it]
Sampling: [20/20]: : 20it [00:57,  2.89s/it]


In [138]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['middl', 'gather', 'fl', 'identifi', 'messag'],
             1: ['contrajeron', 'provincetown', 'cortar', 'ground', 'obstin'],
             2: ['state', 'allow', 'mandat', 'come', 'follow'],
             3: ['vide', 'tez', 'final', 'elt', 'felf'],
             4: ['unnecessari', 'legaci', 'headlin', 'coercion', 'sch'],
             5: ['protein', 'hondur', 'expo', 'cb', 'ne'],
             6: ['diario', 'total', 'dispon', 'dado', 'motorist'],
             7: ['sue', 'oh', 'od', 'em', 'ili'],
             8: ['thrown', 'freedomnotforc', 'trabajador', 'dynam', 'pue'],
             9: ['stori', 'facebook', 'id', 'php', 'post'],
             10: ['pastor', 'biden', 'anticovid', 'muert', 'great'],
             11: ['vacuna', 'deceso', 'salud', 'mil', 'baja'],
             12: ['natur', 'senat', 'especi', 'member', 'ok'],
             13: ['inform', 'like', 'line', 'immun', 'booster'],
             14: ['pm', 'approv', 'hospit', 'like', 'breakthrough'],
   

In [139]:
ctm.get_doc_topic_distribution(training_dataset, n_samples=20)

Sampling: [20/20]: : 20it [00:58,  2.92s/it]


array([[0.02272725, 0.01479618, 0.01936518, ..., 0.02059983, 0.01723773,
        0.02415422],
       [0.02169869, 0.04363298, 0.00830333, ..., 0.07298095, 0.01599228,
        0.01790791],
       [0.00654892, 0.09122271, 0.00228263, ..., 0.00164386, 0.00760538,
        0.00197865],
       ...,
       [0.04880464, 0.02133668, 0.01190491, ..., 0.02941754, 0.02580745,
        0.02406948],
       [0.02438292, 0.01462161, 0.01483011, ..., 0.02462162, 0.02808108,
        0.01401968],
       [0.02372877, 0.00808009, 0.01891906, ..., 0.02035228, 0.03661575,
        0.01800642]])

In [140]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [80565/80565]	Train Loss: 305.76033254398624	Time: 0:00:03.592202: : 15it [00:53,  3.55s/it]
Sampling: [20/20]: : 20it [00:57,  2.89s/it]


In [141]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['biden', 'foxnew', 'muert', 'aunqu', 'increa'],
             1: ['need', 'day', 'right', 'given', 'busi'],
             2: ['org', 'don', 'august', 'know', 'rate'],
             3: ['sa', 'ng', 'like', 'time', 'na'],
             4: ['ito', 'bilang', 'halalan', 'vaccinepassport', 'cardona'],
             5: ['vagi', 'kezel', 'st', 'lj', 'ellen'],
             6: ['ye', 'phone', 'good', 'treatment', 'mask'],
             7: ['parentalauthor', 'richard', 'idaho', 'txag', 'maskchoic'],
             8: ['cooper', 'subscrib', 'yql', 'bot', 'jail'],
             9: ['az', 'hogi', 'sz', 'rt', 'olt'],
             10: ['vacunar', 'lord', 'explica', 'difficulti', 'menor'],
             11: ['experi', 'liberti', 'dose', 'women', 'today'],
             12: ['father', 'pu', 'knew', 'ralli', 'color'],
             13: ['hill', 'da', 'divi', 'parent', 'ramiro'],
             14: ['est', 'epidemiologist', 'stream', 'secretar', 'faster'],
             15: ['lou', 'un

In [142]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [143]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,biden foxnew muert aunqu increa
1,need day right given busi
2,org don august know rate
3,sa ng like time na
4,ito bilang halalan vaccinepassport cardona
5,vagi kezel st lj ellen
6,ye phone good treatment mask
7,parentalauthor richard idaho txag maskchoic
8,cooper subscrib yql bot jail
9,az hogi sz rt olt


### Combined Topic Model

In [144]:
from contextualized_topic_models.models.ctm import CombinedTM
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [80565/80565]	Train Loss: 310.2222491519852	Time: 0:00:04.233071: : 15it [01:02,  4.18s/it]
Sampling: [20/20]: : 20it [01:00,  3.01s/it]


In [145]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['thursday', 'workplac', 'epidemia', 'lan', 'sigu'],
             1: ['school', 'die', 'order', 'individu', 'particip'],
             2: ['speech', 'tra', 'pediatrician', 'biggest', 'polic'],
             3: ['initi', 'price', 'empresa', 'battl', 'navidad'],
             4: ['largest', 'censorship', 'circul', 'ximo', 'dico'],
             5: ['room', 'trigger', 'hondur', 'ayer', 'amid'],
             6: ['protect', 'time', 'shot', 'way', 'legisl'],
             7: ['guess', 'materi', 'advanc', 'pio', 'item'],
             8: ['oh', 'plane', 'evil', 'featur', 'conflict'],
             9: ['state', 'new', 'employ', 'just', 'right'],
             10: ['facebook', 'stori', 'fbid', 'php', 'health'],
             11: ['si', 'hogi', 'nem', 'azt', 'vakcin'],
             12: ['bless', 'googl', 'disclo', 'hondura', 'alcanza'],
             13: ['poblaci', 'pr', 'vacuna', 'utm', 'wed'],
             14: ['premier', 'med', 'guidanc', 'entri', 'australia'],
      

In [146]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [147]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,thursday workplac epidemia lan sigu
1,school die order individu particip
2,speech tra pediatrician biggest polic
3,initi price empresa battl navidad
4,largest censorship circul ximo dico
5,room trigger hondur ayer amid
6,protect time shot way legisl
7,guess materi advanc pio item
8,oh plane evil featur conflict
9,state new employ just right


In [151]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=10, num_epochs=15)
ctm.fit(training_dataset) # run the model
ctm.get_topics(5)

Epoch: [15/15]	 Seen Samples: [80565/80565]	Train Loss: 308.85200741685674	Time: 0:00:04.072360: : 15it [01:02,  4.16s/it]
Sampling: [20/20]: : 20it [00:59,  2.98s/it]


defaultdict(list,
            {0: ['america', 'children', 'nation', 'follow', 'differ'],
             1: ['ceo', 'direcci', 'televicentro', 'mantien', 'protest'],
             2: ['noticia', 'chapintv', 'dan', 'dado', 'bid'],
             3: ['stori', 'health', 'facebook', 'php', 'use'],
             4: ['olt', 'si', 'podcast', 'nueva', 'est'],
             5: ['test', 'caso', 'variant', 'bit', 'center'],
             6: ['gwpihgknrhi', 'gather', 'movi', 'adver', 'passion'],
             7: ['bud', 'lou', 'sick', 'mandat', 'support'],
             8: ['faur', 'rollingston', 'dedicaba', 'pen', 'fyi'],
             9: ['mile', 'provinc', 'text', 'coast', 'congress']})

In [152]:
topics = pd.DataFrame.from_dict(ctm.get_topics(5)).transpose()
topics.columns = ["0", "1", "2" ,"3","4"]

In [153]:
topics['topic_words'] = topics["0"]+ " " + topics["1"]+ " " + topics["2"]+ " " + topics["3"]+ " " + topics["4"]
topics = topics[['topic_words']]
topics

,topic_words
0,america children nation follow differ
1,ceo direcci televicentro mantien protest
2,noticia chapintv dan dado bid
3,stori health facebook php use
4,olt si podcast nueva est
5,test caso variant bit center
6,gwpihgknrhi gather movi adver passion
7,bud lou sick mandat support
8,faur rollingston dedicaba pen fyi
9,mile provinc text coast congress
